In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# *Installing Required Libraries*

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylin

In [ ]:
%cd '/content/drive/MyDrive/'

/content/drive/MyDrive


In [ ]:
!unzip /content/data.zip

Archive:  /content/data.zip
  inflating: data/val.csv            
  inflating: data/train.csv          
  inflating: data/test.csv           
  inflating: data/final_val.csv      
  inflating: data/final_test.csv     
  inflating: data/final_train.csv    


# *Importing libraries*

In [2]:
import json
import pandas as pd
import pickle
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")

train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences'])

# Code for Trans-Literared Text using IndicTrans Library

In [7]:
!git clone https://github.com/libindic/indic-trans.git
%cd /content/indic-trans
!pip install -r requirements.txt
!pip install .

Cloning into 'indic-trans'...
remote: Enumerating objects: 2225, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 2225 (delta 8), reused 10 (delta 3), pack-reused 2206 (from 1)
Receiving objects: 100% (2225/2225), 516.51 MiB | 18.87 MiB/s, done.
Resolving deltas: 100% (1102/1102), done.
Updating files: 100% (719/719), done.
/content/indic-trans
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.4 MB/s eta 0:00:00
Processing /content/indic-trans
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for indictrans: filename=indictrans-1.2.3-cp310-cp310-linux_x86_64.whl size=337821306 sha256=8aebaa89364493095252797bb208d1f8da0b5223ce5ab5c51c30a256226a3ad2
  Stored in directory: /root/.cache/pip/wheels/3e/c9/43/39c5aaa9a570043089bac219a37343294fa5b47f1350dea53a
Successfully bui

In [8]:
!indictrans --h

usage: indictrans [-h] [-v] [-s] [-t] [-b] [-m | -r] [-i] [-o]

Transliterator for Indian Languages including English

options:
  -h, --help          show this help message and exit
  -v, --version       show program's version number and exit
  -s , --source       select language (3 letter ISO-639 code) {hin, guj, pan, ben, mal, kan, tam,
                      tel, ori, eng, mar, nep, bod, kok, asm, urd}
  -t , --target       select language (3 letter ISO-639 code) {hin, guj, pan, ben, mal, kan, tam,
                      tel, ori, eng, mar, nep, bod, kok, asm, urd}
  -b, --build-lookup  build lookup to fasten transliteration
  -m, --ml            use ML system for transliteration
  -r, --rb            use rule-based system for transliteration
  -i , --input        <input-file>
  -o , --output       <output-file>


In [9]:
%cd /content/

/content


In [10]:
from indictrans import Transliterator
trn = Transliterator(source='hin', target='eng', build_lookup=True)
hin = """कांग्रेस पार्टी अध्यक्ष सोनिया गांधी, तमिलनाडु की मुख्यमंत्री
     जयललिता और रिज़र्व बैंक के गवर्नर रघुराम राजन के बीच एक समानता
     है. ये सभी अलग-अलग कारणों से भारतीय जनता पार्टी के राज्यसभा सांसद
     सुब्रमण्यम स्वामी के निशाने पर हैं. उनके जयललिता और सोनिया गांधी के
     पीछे पड़ने का कारण कथित भ्रष्टाचार है."""
eng = trn.transform(hin)
print(eng)

congress party adhyaksh sonia gandhi, tamilnadu kii mukhyamantri
     jayalalita or reserve bank ke governor raghuram rajan ke bich ek samanta
     he. ye sabhi alag-alag kaarnon se bhartiya janata party ke rajyasabha saansad
     subramanyam swami ke nishane par hai. unke jayalalita or sonia gandhi ke
     peeche padane kaa kaaran kathith bhrashtachar he.


In [11]:
trn_en_hin = Transliterator(source='eng', target='hin', build_lookup=True)

In [12]:
import pickle
with open('/content/rh-code-mixed-1.pkl', 'rb') as f:
    related_words = pickle.load(f)

In [13]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [14]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [15]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [16]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [17]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 3060
    })
    val: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 1021
    })
    test: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 1021
    })
})

In [19]:
related_words

{'sarmshar': {'related_words': [{'word': 'sharmshar',
    'sequence_matcher_score': 0.9411764705882353,
    'h_script': 'शर्म्शर्',
    'tag': 'OOV'}],
  'h_script': 'सर्मशार',
  'corrected_word': 'sarma share',
  'language': 'EN',
  'check': 1},
 'laut': {'related_words': [],
  'h_script': 'लौत',
  'language': 'HI',
  'check': 1},
 'andhjihadi': {'related_words': [],
  'h_script': 'अन्ध्जिहदि',
  'corrected_word': 'andha jihaadi',
  'language': 'HI',
  'check': 1},
 'wadi': {'related_words': [], 'h_script': 'वदि', 'language': 'HI'},
 'theazad': {'related_words': [],
  'h_script': 'थेअज़द्',
  'corrected_word': 'the azad',
  'language': 'HI',
  'check': 1},
 'rahena': {'related_words': [{'word': 'rahenga',
    'sequence_matcher_score': 0.9230769230769231,
    'h_script': 'रहेन्ग',
    'tag': 'HI'}],
  'h_script': 'रहेना',
  'language': 'HI',
  'check': 1},
 'gdari': {'related_words': [], 'h_script': 'ग्दरि', 'language': 'HI'},
 'kep': {'related_words': [],
  'h_script': 'केप',
  'correc

In [20]:
for key, value in related_words.items():
    # Use the corrected word if present, else use the key
    corrected_word = value.get('corrected_word', key)
    if corrected_word:
        # Transliterate to Devanagari script
        transliterated_word = trn_en_hin.transform(corrected_word)
        print(">>>>>>",transliterated_word)
        related_words[key]['h_script'] = transliterated_word
    else:
        corrected_word = key
        transliterated_word = trn_en_hin.transform(corrected_word)
        print("??????",corrected_word)
        related_words[key]['h_script'] = transliterated_word

Streaming output truncated to the last 5000 lines.
>>>>>> हमेरे
>>>>>> दीवाल इसीसी
>>>>>> देध
>>>>>> फॉज
>>>>>> हम्षा
>>>>>> 98
>>>>>> बोलोगे
>>>>>> बताये
>>>>>> बंगल्देशी
>>>>>> मनने वालों
>>>>>> धन्यवाद
>>>>>> स्टार
>>>>>> नज़रों
>>>>>> तंगवा
>>>>>> भौंके
>>>>>> रोये
>>>>>> बंगले
>>>>>> चाहिय
>>>>>> कस्टे
>>>>>> बेसहारा
>>>>>> मिट
>>>>>> पहले
>>>>>> तुघे
>>>>>> कन्नो हलाला
>>>>>> चुनते
>>>>>> रप
>>>>>> खिल्फ़त
>>>>>> पाऊँग्य
>>>>>> दिखाते
>>>>>> टाइम्स
>>>>>> भीयजी
>>>>>> पुण्य
>>>>>> तेयार
>>>>>> नवाब
>>>>>> विचर
>>>>>> समजदर
>>>>>> ट्रास्ट
>>>>>> अंबाला
>>>>>> सैनिक
>>>>>> वा
>>>>>> अटके
>>>>>> गाथर
>>>>>> लगें
>>>>>> पड़ी
>>>>>> दीवार
>>>>>> फुले
>>>>>> द्र
>>>>>> कुरियाँ
>>>>>> शॉपर
>>>>>> सिटीज
>>>>>> फिनेड
>>>>>> ज्योतिका
>>>>>> धिखाया
>>>>>> समल
>>>>>> अम्बेडकर
>>>>>> ब्लिंड
>>>>>> दुआ
>>>>>> प्रोटेक्टिव
>>>>>> गरेबो
>>>>>> कबिलियट
>>>>>> चिजें
>>>>>> इज्जत भरवा
>>>>>> प्रभु
>>>>>> सका
>>>>>> गुरुकुल
>>>>>> केजिरवाल
>>>>>> बêते
>>>>>> उदहारण
>>>>>> नेपाली
>>>>>> मैनो
>>>>>> पह

## Transliterated Word language with Devanagri Hindi scipt and english words using language tags Approach

In [21]:
def transliterated_preprocess(sentences, related_words):
    texts = sentences['clean_text']
    processed_texts = []

    for text in texts:
        words = text.split()
        process_text = []

        for word in words:
            # Check if the word exists in the related_words dictionary
            if word in related_words and related_words[word].get('language') == 'HI':
                # Append the h_script value from related_words to the interleaved_text list
                process_text.append(related_words[word]['h_script'])
            else:
                # If the word is not in related_words, just append the original word
                process_text.append(word)

        # Join the list back into a string
        processed_texts.append(" ".join(process_text))

    return {"transliterated_text_dev": processed_texts}

dataset = dataset.map(
    transliterated_preprocess,
    fn_kwargs={'related_words': related_words},
    batched=True
)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [24]:
print(dataset['train'][0]['transliterated_text_dev'])

hindu तो कुछ हैं पर तुम तो पोरी दुनिया मैं गंदगी मचा रे हो or bat री भगवे की तो तुम्हे aba के अबे भगवा ही थे तलवार के दर से सल्वार खोल दी उन्होंने इसमें तुम्हारी कोई गलती न्ही हैं


In [22]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'corrected_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvaar khol di unhone isme tumhari koi galti nhi hai',
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI',
 'transliterated_text_dev': 'hindu तो कुछ हैं पर तुम तो पोरी दुनिया मैं गंदगी मचा रे हो or bat री भगवे की तो तुम्हे aba के अबे भगवा ही थे तलवार के दर से सल्वार खोल दी उन्होंने इसमें तुम्हारी कोई गलती न्ही हैं'}

In [23]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'corrected_text': 'hindu rastra ke rujhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI',
 'transliterated_text_dev': 'hindu rastra के रुझन मियाँ भाई को कस्त हैं चलो hindu इस bat पे mast हैं'}

In [25]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'corrected_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI',
 'transliterated_text_dev': 'जय hind , in नमुनो को bat भी करनी नहीं अति हैं , इनको pension चाहिए'}

# For 5000 rows dataset with clean_text

# *Tokenization as per Hing-MBERT*

In [26]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
print(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

119547


In [27]:
len(tokenizer)

119547

In [28]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['transliterated_text_dev'],max_length=97,padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","corrected_text","language_tags","transliterated_text_dev"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [29]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  19911,  11460,  21042,  22022,  11716,  12213,    880,  14070,
          13841,  21042,    885,  41937,  10914,  83874,    889,  99007,    867,
          52768,  34315,    889,  22078,    891,  11554,  13220,  10345,  11519,
            891,  10914,    888,  19741,  72109,  10826,  21042,    880,  14070,
          45753, 103860,  11357,  10113,  10412,  49545,  11554,    888,  19741,
          28960,  14080,  17798,    880,  11714,  52884,  10412, 100906,  11072,
            898,  11714,  27155,  19885,    866,  51140,  41607,  27640,  53744,
            880,  14070,  45753,  42263,  10914,  38207,    867,  11714,  18406,
            884,  99386,  11716,    102,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [31]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'hin', '##du', 'तो', 'कुछ', 'हैं', 'पर', 'त', '##ु', '##म', 'तो', 'प', '##ोर', '##ी', 'दुनिया', 'म', '##ैं', 'ग', '##ंद', '##गी', 'म', '##चा', 'र', '##े', 'हो', 'or', 'bat', 'र', '##ी', 'भ', '##ग', '##वे', 'की', 'तो', 'त', '##ु', '##म्', '##हे', 'ab', '##a', 'के', 'अब', '##े', 'भ', '##ग', '##वा', 'ही', 'थे', 'त', '##ल', '##वार', 'के', 'दर', 'से', 'स', '##ल', '##्व', '##ार', 'ख', '##ोल', 'दी', 'उन्होंने', 'इसमें', 'त', '##ु', '##म्', '##हार', '##ी', 'कोई', 'ग', '##ल', '##ती', 'न', '##्ही', 'हैं', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [30]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

input_str = dataset['train'][0]['transliterated_text_dev']
# Tokenization happens in a few steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

start:                 hindu तो कुछ हैं पर तुम तो पोरी दुनिया मैं गंदगी मचा रे हो or bat री भगवे की तो तुम्हे aba के अबे भगवा ही थे तलवार के दर से सल्वार खोल दी उन्होंने इसमें तुम्हारी कोई गलती न्ही हैं
tokenize:              ['hin', '##du', 'तो', 'कुछ', 'हैं', 'पर', 'त', '##ु', '##म', 'तो', 'प', '##ोर', '##ी', 'दुनिया', 'म', '##ैं', 'ग', '##ंद', '##गी', 'म', '##चा', 'र', '##े', 'हो', 'or', 'bat', 'र', '##ी', 'भ', '##ग', '##वे', 'की', 'तो', 'त', '##ु', '##म्', '##हे', 'ab', '##a', 'के', 'अब', '##े', 'भ', '##ग', '##वा', 'ही', 'थे', 'त', '##ल', '##वार', 'के', 'दर', 'से', 'स', '##ल', '##्व', '##ार', 'ख', '##ोल', 'दी', 'उन्होंने', 'इसमें', 'त', '##ु', '##म्', '##हार', '##ी', 'कोई', 'ग', '##ल', '##ती', 'न', '##्ही', 'हैं']
convert_tokens_to_ids: [19911, 11460, 21042, 22022, 11716, 12213, 880, 14070, 13841, 21042, 885, 41937, 10914, 83874, 889, 99007, 867, 52768, 34315, 889, 22078, 891, 11554, 13220, 10345, 11519, 891, 10914, 888, 19741, 72109, 10826, 21042, 880, 14070, 45753, 103860, 11357,

# *Calculating Weights for Labels with Imbalanced Dataset*

In [32]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [33]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [34]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [35]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [36]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)

model.resize_token_embeddings(len(tokenizer))

arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k_trandev",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }


trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [37]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [38]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k_trandev/log.jsonl"))

In [39]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.685893,0.759060,0.721173,0.759060,0.659657,0.759060,0.759060,0.504781,0.443190,0.012146,0.023810,0.997416,0.862570
2,No log,0.668271,0.470127,0.746117,0.470127,0.482715,0.470127,0.470127,0.605033,0.468749,0.866397,0.441692,0.343669,0.495806
3,No log,0.594728,0.752204,0.763177,0.752204,0.756988,0.752204,0.752204,0.683572,0.675659,0.550607,0.518095,0.816537,0.833223
4,No log,0.610668,0.715965,0.758096,0.715965,0.730252,0.715965,0.715965,0.683102,0.656435,0.619433,0.513423,0.746770,0.799447
5,No log,0.603531,0.675808,0.763730,0.675808,0.698120,0.675808,0.675808,0.688317,0.635906,0.712551,0.515373,0.664083,0.756439
6,No log,0.648560,0.689520,0.776685,0.689520,0.710888,0.689520,0.689520,0.707009,0.651306,0.740891,0.535871,0.673127,0.766740
7,No log,0.790517,0.729677,0.763829,0.729677,0.741767,0.729677,0.729677,0.690767,0.667678,0.615385,0.524138,0.766150,0.811218


TrainOutput(global_step=336, training_loss=0.524800663902646, metrics={'train_runtime': 635.6774, 'train_samples_per_second': 48.138, 'train_steps_per_second': 0.755, 'train_loss': 0.524800663902646, 'epoch': 7.0})

In [40]:
mbert_results = trainer.evaluate()

In [41]:
print(mbert_results)

{'eval_loss': 0.5947277545928955, 'eval_accuracy': 0.752203721841332, 'eval_precision_weighted': 0.7631765990300023, 'eval_recall_weighted': 0.752203721841332, 'eval_f1_weighted': 0.7569877449106325, 'eval_recall_micro': 0.752203721841332, 'eval_f1_micro': 0.752203721841332, 'eval_recall_macro': 0.6835723775748255, 'eval_f1_macro': 0.6756593527325234, 'eval_recall_positive': 0.5506072874493927, 'eval_f1_positive': 0.5180952380952382, 'eval_recall_negative': 0.8165374677002584, 'eval_f1_negative': 0.8332234673698088, 'eval_runtime': 5.4427, 'eval_samples_per_second': 187.591, 'eval_steps_per_second': 11.759, 'epoch': 7.0}


# **Hing-MBERT Test results**

In [42]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [43]:
test_results_mbert

PredictionOutput(predictions=array([[ 0.65595186, -0.83605564],
       [ 0.33539537, -0.57610166],
       [ 0.39812276, -0.67025334],
       ...,
       [ 0.04980446, -0.17933162],
       [-0.5344889 ,  0.4700711 ],
       [ 0.23381463, -0.5061584 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.5839678645133972, 'test_accuracy': 0.7482859941234085, 'test_precision_weighted': 0.7662083615920287, 'test_recall_weighted': 0.7482859941234085, 'test_f1_weighted': 0.7555291844217183, 'test_recall_micro': 0.7482859941234085, 'test_f1_micro': 0.7482859941234085, 'test_recall_macro': 0.6906364749082008, 'test_f1_macro': 0.6776279336623433, 'test_recall_positive': 0.5789473684210527, 'test_f1_positive': 0.5267034990791897, 'test_recall_negative': 0.8023255813953488, 'test_f1_negative': 0.8285523682454969, 'test_runtime': 5.4317, 'test_samples_per_second': 187.972, 'test_steps_per_second': 11.783})

In [44]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 0, 0, ..., 0, 1, 0])

In [45]:
test_df['trandev_predicted_labels_5k_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [46]:
print(test_results_mbert.metrics)

{'test_loss': 0.5839678645133972, 'test_accuracy': 0.7482859941234085, 'test_precision_weighted': 0.7662083615920287, 'test_recall_weighted': 0.7482859941234085, 'test_f1_weighted': 0.7555291844217183, 'test_recall_micro': 0.7482859941234085, 'test_f1_micro': 0.7482859941234085, 'test_recall_macro': 0.6906364749082008, 'test_f1_macro': 0.6776279336623433, 'test_recall_positive': 0.5789473684210527, 'test_f1_positive': 0.5267034990791897, 'test_recall_negative': 0.8023255813953488, 'test_f1_negative': 0.8285523682454969, 'test_runtime': 5.4317, 'test_samples_per_second': 187.972, 'test_steps_per_second': 11.783}


# **Hing BERT Training**

In [47]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [48]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['transliterated_text_dev'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","corrected_text","language_tags","transliterated_text_dev"])
bert_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [49]:
## model training
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

bert_model.resize_token_embeddings(len(bert_tokenizer))
arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k_trandev",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k_trandev/log.jsonl"))

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [50]:
# train the model
bert_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.688611,0.758080,0.702385,0.758080,0.681022,0.758080,0.758080,0.523431,0.490368,0.068826,0.120996,0.978036,0.859739
2,No log,0.643741,0.703232,0.718385,0.703232,0.709938,0.703232,0.703232,0.620950,0.614425,0.461538,0.429379,0.780362,0.799471
3,No log,0.618657,0.754163,0.749823,0.754163,0.751864,0.754163,0.754163,0.655920,0.658688,0.465587,0.478170,0.846253,0.839206
4,No log,0.614772,0.714985,0.739489,0.714985,0.724802,0.714985,0.714985,0.653511,0.639990,0.534413,0.475676,0.772610,0.804304
5,No log,0.638338,0.667973,0.730828,0.667973,0.688050,0.667973,0.667973,0.643178,0.611929,0.595142,0.464455,0.691214,0.759404


TrainOutput(global_step=240, training_loss=0.6204070409138998, metrics={'train_runtime': 356.1587, 'train_samples_per_second': 85.917, 'train_steps_per_second': 1.348, 'train_loss': 0.6204070409138998, 'epoch': 5.0})

In [51]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.6147722005844116, 'eval_accuracy': 0.7149853085210578, 'eval_precision_weighted': 0.7394891608028635, 'eval_recall_weighted': 0.7149853085210578, 'eval_f1_weighted': 0.7248023143069141, 'eval_recall_micro': 0.7149853085210578, 'eval_f1_micro': 0.7149853085210578, 'eval_recall_macro': 0.653511387293517, 'eval_f1_macro': 0.6399898216979589, 'eval_recall_positive': 0.5344129554655871, 'eval_f1_positive': 0.4756756756756757, 'eval_recall_negative': 0.772609819121447, 'eval_f1_negative': 0.8043039677202422, 'eval_runtime': 5.3266, 'eval_samples_per_second': 191.68, 'eval_steps_per_second': 12.015, 'epoch': 5.0}


# **Test results for Hing-BERT**

In [52]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['transdev_predicted_labels_5k_hingBert'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 0.6226420402526855, 'test_accuracy': 0.703232125367287, 'test_precision_weighted': 0.7352309699023449, 'test_recall_weighted': 0.703232125367287, 'test_f1_weighted': 0.7154746600882063, 'test_recall_micro': 0.703232125367287, 'test_f1_micro': 0.703232125367287, 'test_recall_macro': 0.6485160426408896, 'test_f1_macro': 0.6316848445307468, 'test_recall_positive': 0.5425101214574899, 'test_f1_positive': 0.46935201401050786, 'test_recall_negative': 0.7545219638242894, 'test_f1_negative': 0.7940176750509859, 'test_runtime': 5.4552, 'test_samples_per_second': 187.16, 'test_steps_per_second': 11.732}


In [ ]:
test_df.to_csv('final_test_5k_itags.csv',index=False, encoding='utf-8')

# **Muril cased training**

In [53]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [54]:
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
print(len(muril_tokenizer))

muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['transliterated_text_dev'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)

muril_dataset = muril_dataset.remove_columns(["clean_text","corrected_text","language_tags","transliterated_text_dev"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)

muril_model.resize_token_embeddings(len(muril_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k_trandev",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=3e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k_trandev/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

197285


Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [55]:
muril_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.679424,0.745348,0.763803,0.745348,0.752801,0.745348,0.745348,0.687320,0.674247,0.574899,0.522059,0.799742,0.826435
2,No log,0.623834,0.794319,0.777126,0.794319,0.779500,0.794319,0.794319,0.664488,0.681880,0.412955,0.492754,0.916021,0.871007
3,0.645500,0.595603,0.721841,0.778456,0.721841,0.738337,0.721841,0.721841,0.713165,0.673461,0.696356,0.547771,0.729974,0.799151
4,0.645500,0.593246,0.767875,0.790964,0.767875,0.776283,0.767875,0.767875,0.728366,0.708109,0.651822,0.576029,0.804910,0.840189
5,0.645500,0.726232,0.795299,0.785954,0.795299,0.789401,0.795299,0.795299,0.695457,0.705404,0.502024,0.542670,0.888889,0.868139
6,0.459500,0.771589,0.795299,0.782650,0.795299,0.786386,0.795299,0.795299,0.684430,0.697767,0.469636,0.526077,0.899225,0.869457
7,0.459500,0.813325,0.759060,0.778365,0.759060,0.766577,0.759060,0.759060,0.708769,0.693218,0.611336,0.551095,0.806202,0.835341


TrainOutput(global_step=1344, training_loss=0.489095790045602, metrics={'train_runtime': 837.4179, 'train_samples_per_second': 36.541, 'train_steps_per_second': 2.293, 'train_loss': 0.489095790045602, 'epoch': 7.0})

In [56]:
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

{'eval_loss': 0.593245804309845, 'eval_accuracy': 0.7678746327130265, 'eval_precision_weighted': 0.7909637504276669, 'eval_recall_weighted': 0.7678746327130265, 'eval_f1_weighted': 0.7762832575688797, 'eval_recall_micro': 0.7678746327130265, 'eval_f1_micro': 0.7678746327130265, 'eval_recall_macro': 0.7283657115358462, 'eval_f1_macro': 0.7081087145068077, 'eval_recall_positive': 0.6518218623481782, 'eval_f1_positive': 0.5760286225402506, 'eval_recall_negative': 0.8049095607235142, 'eval_f1_negative': 0.8401888064733647, 'eval_runtime': 5.3494, 'eval_samples_per_second': 190.864, 'eval_steps_per_second': 11.964, 'epoch': 7.0}


In [57]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test_df['transdev_predicted_labels_5k_muril'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

{'test_loss': 0.609022319316864, 'test_accuracy': 0.7482859941234085, 'test_precision_weighted': 0.7725176673035046, 'test_recall_weighted': 0.7482859941234085, 'test_f1_weighted': 0.7574042075746924, 'test_recall_micro': 0.7482859941234085, 'test_f1_micro': 0.7482859941234085, 'test_recall_macro': 0.7016628482356757, 'test_f1_macro': 0.683476538515821, 'test_recall_positive': 0.611336032388664, 'test_f1_positive': 0.5402504472271914, 'test_recall_negative': 0.7919896640826873, 'test_f1_negative': 0.8267026298044505, 'test_runtime': 5.4444, 'test_samples_per_second': 187.532, 'test_steps_per_second': 11.755}


## XLMR training

In [58]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [59]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['transliterated_text_dev'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","corrected_text","language_tags","transliterated_text_dev"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k_transdev",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k_transdev/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import torch
torch.cuda.empty_cache()

In [60]:
xlmr_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.647071,0.763957,0.745852,0.763957,0.751805,0.763957,0.763957,0.636192,0.646818,0.388664,0.443418,0.883721,0.850218
2,No log,0.597930,0.664055,0.774161,0.664055,0.688075,0.664055,0.664055,0.698483,0.632310,0.765182,0.524272,0.631783,0.740348
3,0.635700,0.584705,0.748286,0.791285,0.748286,0.761503,0.748286,0.748286,0.731985,0.697601,0.700405,0.573798,0.763566,0.821404
4,0.635700,0.639877,0.702253,0.778414,0.702253,0.722002,0.702253,0.702253,0.711272,0.660780,0.728745,0.542169,0.693798,0.779390
5,0.635700,0.779116,0.777669,0.789140,0.777669,0.782452,0.777669,0.777669,0.721043,0.710428,0.611336,0.570888,0.830749,0.849967
6,0.456900,0.796839,0.789422,0.791489,0.789422,0.790415,0.789422,0.789422,0.717768,0.715669,0.578947,0.570858,0.856589,0.860480
7,0.456900,1.020989,0.782566,0.792820,0.782566,0.786885,0.782566,0.782566,0.725651,0.715754,0.615385,0.577947,0.835917,0.853562


TrainOutput(global_step=1344, training_loss=0.48173224925994873, metrics={'train_runtime': 1286.9935, 'train_samples_per_second': 23.776, 'train_steps_per_second': 1.492, 'train_loss': 0.48173224925994873, 'epoch': 7.0})

In [61]:
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 0.5847054719924927, 'eval_accuracy': 0.7482859941234085, 'eval_precision_weighted': 0.7912845989893574, 'eval_recall_weighted': 0.7482859941234085, 'eval_f1_weighted': 0.761502968708151, 'eval_recall_micro': 0.7482859941234085, 'eval_f1_micro': 0.7482859941234085, 'eval_recall_macro': 0.7319853748862317, 'eval_f1_macro': 0.6976007154406332, 'eval_recall_positive': 0.7004048582995951, 'eval_f1_positive': 0.5737976782752902, 'eval_recall_negative': 0.7635658914728682, 'eval_f1_negative': 0.8214037526059763, 'eval_runtime': 11.3749, 'eval_samples_per_second': 89.759, 'eval_steps_per_second': 5.626, 'epoch': 7.0}


In [62]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test_df['trandev_predicted_labels_5k_xlmr'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

{'test_loss': 0.5861614346504211, 'test_accuracy': 0.7335945151811949, 'test_precision_weighted': 0.7845846958836444, 'test_recall_weighted': 0.7335945151811949, 'test_f1_weighted': 0.7487221638205231, 'test_recall_micro': 0.7335945151811949, 'test_f1_micro': 0.7335945151811949, 'test_recall_macro': 0.7222954524056115, 'test_f1_macro': 0.6844296571033781, 'test_recall_positive': 0.7004048582995951, 'test_f1_positive': 0.5598705501618123, 'test_recall_negative': 0.7441860465116279, 'test_f1_negative': 0.8089887640449438, 'test_runtime': 11.7871, 'test_samples_per_second': 86.62, 'test_steps_per_second': 5.43}


In [63]:
test_df.to_csv('final_test_5k_transdev_v2.csv',index=False, encoding='utf-8')